In [1]:
'''
Objetivo é treinar um modelo de regressão linear para fazer previsões de preços das empresa Sony, Microsoft, Ubisoft, Nintendo, Eletronic Arts

'''

'\nObjetivo é treinar um modelo de regressão linear para fazer previsões de preços das empresa Sony, Microsoft, Ubisoft, Nintendo, Eletronic Arts\n\n'

In [2]:
#Bibliotecas utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import datetime as dt

In [3]:
#Carregamento dos dados

dados =  pd.read_excel(r"C:\Users\dante\OneDrive\Documentos\9. Meus Projetos\Projetos Power Bi\Mercado de Ações\Dados\Agrupado.xlsx")

df = pd.DataFrame(dados)

In [4]:
#Visão Geral

print(df.shape)
print(df.columns)
print(df.head())
df.info()

#A coluna 'Date' será a variavel preditora
#As colunas 'Close/Last', 'Volume', 'Open', 'High', 'Low' serão as colunas alvos

(6290, 7)
Index(['Empresa', 'Date', 'Close/Last', 'Volume', 'Open', 'High', 'Low'], dtype='object')
          Empresa                 Date Close/Last   Volume     Open      High  \
0  Eletronic Arts  2024-04-11 00:00:00    $152.89  1975349  $152.16   $153.09   
1  Eletronic Arts  2024-01-11 00:00:00    $151.26  1954157  $150.83   $152.46   
2  Eletronic Arts           10/31/2024    $150.85  2645877  $148.31  $151.585   
3  Eletronic Arts           10/30/2024    $149.14  3600584  $147.48   $151.34   
4  Eletronic Arts           10/29/2024    $145.62  2703361  $144.14   $146.26   

        Low  
0   $150.40  
1   $149.14  
2   $147.59  
3   $144.68  
4  $144.095  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6290 entries, 0 to 6289
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Empresa     6290 non-null   object
 1   Date        6290 non-null   object
 2   Close/Last  6290 non-null   object
 3   Volume      6290 n

In [5]:
#Salvando uma cópia do dataset

df_original = df.copy()

In [6]:
#Conversão de objeto para data
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format=None)
#conferindo as informações
print(df['Date'].dtypes)
print(df['Date'].head())



datetime64[ns]
0   2024-04-11
1   2024-01-11
2   2024-10-31
3   2024-10-30
4   2024-10-29
Name: Date, dtype: datetime64[ns]


In [7]:
#tipos das colunas

tipos = df.dtypes
print(tipos)

Empresa               object
Date          datetime64[ns]
Close/Last            object
Volume                 int64
Open                  object
High                  object
Low                   object
dtype: object


In [8]:
#Análise Exploratória - Resumo Estatístico

valores_ausentes = df.isnull().sum()
print(valores_ausentes)


Empresa       0
Date          0
Close/Last    0
Volume        0
Open          0
High          0
Low           0
dtype: int64


In [9]:
#Tirando o cifrão do número

df['Open'] = df['Open'].str.replace("$", '')
df['Close/Last'] = df['Close/Last'].str.replace("$", '')
df['High'] = df['High'].str.replace("$", '')
df['Low'] = df['Low'].str.replace("$", '')

print(df['Open'].head())
print(df['Close/Last'].head())
print(df['High'].head())
print(df['Low'].head())

0    152.16
1    150.83
2    148.31
3    147.48
4    144.14
Name: Open, dtype: object
0    152.89
1    151.26
2    150.85
3    149.14
4    145.62
Name: Close/Last, dtype: object
0     153.09
1     152.46
2    151.585
3     151.34
4     146.26
Name: High, dtype: object
0     150.40
1     149.14
2     147.59
3     144.68
4    144.095
Name: Low, dtype: object


In [10]:
#Mudando o tipo de objeto para float
df['Open'] = df['Open'].astype(float)
df['Close/Last'] = df['Close/Last'].astype(float)
df['High'] = df['High'].astype(float)
df['Low'] = df['Low'].astype(float)
df['Volume'] = df['Volume'].astype(float)
print(df.dtypes)

Empresa               object
Date          datetime64[ns]
Close/Last           float64
Volume               float64
Open                 float64
High                 float64
Low                  float64
dtype: object


In [11]:

descricao_vol = df['Volume'].describe().round()
descricao_open = df['Open'].describe().round()
descricao_high = df['High'].describe().round()
descricao_low = df['Low'].describe().round()
descricao_close = df['Close/Last'].describe().round()
print(descricao_vol)

count        6290.0
mean      7425999.0
std      12179577.0
min           905.0
25%        799419.0
50%       2295472.0
75%       5590171.0
max      97073560.0
Name: Volume, dtype: float64


In [12]:
print(df['Date'].head())

0   2024-04-11
1   2024-01-11
2   2024-10-31
3   2024-10-30
4   2024-10-29
Name: Date, dtype: datetime64[ns]


In [13]:
#verificando primeiro valor da coluna 'Date'

primeiro_linha = df.sort_values('Date').iloc[0]
print(primeiro_linha)

Empresa       Nintendo Corporation
Date           2019-02-12 00:00:00
Close/Last                  10.008
Volume                   1959825.0
Open                         9.876
High                         10.16
Low                          9.876
Name: 3755, dtype: object


In [14]:
#transformando a coluna 'Empresa' do tipo string
df['Empresa'] = df['Empresa'].astype(str)

In [15]:
#filtrando as empresas

empresa = df['Empresa'].unique()

sony_dados = df[df['Empresa'] == 'Sony Corporation']
micro_dados = df[df['Empresa'] == 'Microsoft Corporation']
ubi_dados = df[df['Empresa'] == 'Ubsoft Entertainment']
dados_EA = df[df['Empresa'] == 'Eletronic Arts']


In [16]:
#SONY
#pegando a data de inicio
min_data = sony_dados['Date'].min()

sony_dados['dias_desde_inicio'] = (sony_dados['Date'] - min_data).dt.days

print(sony_dados['dias_desde_inicio'].head())



3774    1885
3775    1794
3776    2088
3777    2087
3778    2086
Name: dias_desde_inicio, dtype: int64


C:\Users\dante\AppData\Local\Temp\ipykernel_13576\1207929568.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sony_dados['dias_desde_inicio'] = (sony_dados['Date'] - min_data).dt.days


In [17]:
#SONY
#Criação das variaveis features e Targets
X_sony = sony_dados[['dias_desde_inicio', 'High', 'Low', 'Open']] 
y_sony = sony_dados['Close/Last']


In [18]:
#SONY
# Treinando o Modelo

x_train, x_test, y_train, y_test = train_test_split(X_sony, y_sony, test_size=0.2, random_state=42)

In [19]:
#SONY
# Treinar o modelo de regressão linear
modelo = LinearRegression()
modelo.fit(x_train, y_train)

LinearRegression()

In [20]:
#SONY
#SALVAR ESTA VARIAVEL
# Fazer previsões
y_pred = modelo.predict(x_test)

In [21]:
# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [39]:
print('Período do dataset: ')
print(df['Date'].min())
print(df['Date'].max())

Período do dataset: 
2019-02-12 00:00:00
2024-12-09 00:00:00


In [22]:
#Microsoft

min_data_micro = micro_dados['Date'].min()

micro_dados['data_inicio'] = (micro_dados['Date'] - min_data_micro).dt.days

print(micro_dados['data_inicio'].head())


1258    1885
1259    1794
1260    2088
1261    2087
1262    2086
Name: data_inicio, dtype: int64


C:\Users\dante\AppData\Local\Temp\ipykernel_13576\3301019721.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  micro_dados['data_inicio'] = (micro_dados['Date'] - min_data_micro).dt.days


In [23]:
#Microsoft Target

#Criação das variaveis features e Targets
X_micro = micro_dados[['data_inicio', 'High', 'Low', 'Open']] 
y_micro = micro_dados['Close/Last']

In [24]:
#Microsoft
# Treinando o Modelo

x_train_micro, x_test_micro, y_train_micro, y_test_micro = train_test_split(X_micro, y_micro, test_size=0.2, random_state=42)

In [25]:
#Microsoft
# Treinar o modelo de regressão linear
modelo_micro = LinearRegression()
modelo_micro.fit(x_train_micro, y_train_micro)

LinearRegression()

In [26]:
#Microsoft
#SALVAR ESTA VARIAVEL
# Fazer previsões
y_pred_micro = modelo_micro.predict(x_test_micro)

print(y_pred_micro)

[287.6982554  431.66608645 418.51348481 403.720683   202.854292
 250.79260583 299.09266342 212.98521734 167.57600348 201.21127457
 239.60027822 226.58887597 397.57949937 278.50495819 210.17515097
 326.48926457 203.29827186 302.93901703 284.25418144 215.68134852
 213.13973455 254.56843035 372.28155974 310.54962965 215.23486342
 332.20905148 264.3173817  328.83725102 280.91814632 409.36434628
 231.83493553 354.81504906 176.84020398 250.10039845 252.36615709
 247.27682932 323.15517052 328.7871088  240.77111967 405.43828113
 160.16113324 284.66153738 240.22647786 411.45301811 286.44770424
 239.3443081  328.16213836 418.30746189 243.91742834 268.503708
 238.63023211 335.70370566 285.9744485  253.36088435 377.6135859
 369.03133278 374.10499177 405.19836279 185.00023745 306.95566749
 251.11528006 318.17653635 251.17944299 455.30119159 414.31839732
 136.18533184 184.87052681 448.43324644 270.56076084 413.2570738
 240.31631078 436.69045895 311.53949523 189.47576857 282.39577623
 184.93650728 34

In [37]:
# Obter as datas correspondentes ao conjunto de teste
datas_teste = x_test_micro['data_inicio']

# Exibir o intervalo de datas do conjunto de teste
print("Intervalo de datas do conjunto de teste:")
print(datas_teste.min(), "a", datas_teste.max())


Intervalo de datas do conjunto de teste:
274 a 2120


In [27]:
#Ubisoft
min_data_ubi = ubi_dados['Date'].min()

ubi_dados['data_inicio_ubi'] = (ubi_dados['Date'] - min_data_ubi).dt.days

C:\Users\dante\AppData\Local\Temp\ipykernel_13576\4086916052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ubi_dados['data_inicio_ubi'] = (ubi_dados['Date'] - min_data_ubi).dt.days


In [28]:
#Ubisoft targe

#Criação das variaveis features e Targets
X_ubi = ubi_dados[['data_inicio_ubi', 'High', 'Low', 'Open']] 
y_ubi = ubi_dados['Close/Last']

In [29]:
#Ubisoft
# Treinando o Modelo

x_train_ubi, x_test_ubi, y_train_ubi, y_test_ubi = train_test_split(X_ubi, y_ubi, test_size=0.2, random_state=42)

In [30]:
#Ubisoft
# Treinar o modelo de regressão linear
modelo_ubi = LinearRegression()
modelo_ubi.fit(x_train_ubi, y_train_ubi)

LinearRegression()

In [31]:
#Ubisoft
#SALVAR ESTA VARIAVEL
# Fazer previsões
y_pred_ubi = modelo_ubi.predict(x_test_ubi)

print(y_pred_ubi)

[ 9.50441025  4.71697566  3.69066083  3.70570616 16.41782274  4.39032663
 13.07973954 16.31472656 14.46481274 17.09822268  5.98160462 16.58598032
  4.37318213  8.45286609 16.42857464  5.75641617 16.06283522 10.58062328
  5.22335515 18.67993894 18.22558093  4.76034291  5.25988143 10.47748047
 16.42218807  9.80453035  9.32760746  5.63170347  8.91650303  3.41780128
 16.658076    6.16028006 14.36208557 15.79163079  4.20576587  5.64118734
  6.09340536  6.0258594   5.70080295  4.35795053 13.94941082  9.67575041
 20.09418302  4.75537823 12.17060634 15.69596116  6.03894925  2.25302113
  5.08730637 13.86508493  4.1100531   5.61156158 12.81911573 15.19858115
  5.64014024  6.46401936  4.94242392  4.62247831 16.0668734   5.86064278
  7.02184462  5.9081809   4.28385532  4.80905867  3.70783108 13.211
 15.18594911  4.74004516  4.76522124  3.81132077 15.45627532  4.63847149
  5.75161688 15.2123401   8.43636737 15.30909065 10.57626151  5.83148316
  4.31794171 11.46360824  5.49712212  5.69671293  6.0309

In [32]:
#Electronic Arts dados_EA

min_data_EA = dados_EA['Date'].min()

dados_EA['data_inicio_EA'] = (dados_EA['Date'] - min_data_EA).dt.days

C:\Users\dante\AppData\Local\Temp\ipykernel_13576\1191675173.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_EA['data_inicio_EA'] = (dados_EA['Date'] - min_data_EA).dt.days


In [33]:
#Electronic Arts target

#Criação das variaveis features e Targets
X_EA = dados_EA[['data_inicio_EA', 'High', 'Low', 'Open']] 
y_EA = dados_EA['Close/Last']

In [34]:
#Electronic Arts
# Treinando o Modelo

x_train_EA, x_test_EA, y_train_EA, y_test_EA = train_test_split(X_EA, y_EA, test_size=0.2, random_state=42)

In [35]:
#Electronic Arts
# Treinar o modelo de regressão linear
modelo_EA = LinearRegression()
modelo_EA.fit(x_train_EA, y_train_EA)

LinearRegression()

In [36]:
#Electronic Arts
#SALVAR ESTA VARIAVEL
# Fazer previsões
y_pred_EA = modelo_EA.predict(x_test_EA)

print(y_pred_EA)

[130.42572783 136.52036575 147.9537627  146.92929718 137.83316586
 110.97620632 142.21771258 127.62706928 114.0070462  127.3916792
 117.60517547 138.13848966 138.87410003 128.48735884 142.15262565
 126.98259253 136.79754729 134.60358003 126.59929917 137.27048538
 139.6229618  109.86005911 137.57309436 142.99083638 141.63427054
 126.23928199 125.17537989 127.49332217 119.43254266 147.51444204
 138.84347157 137.82102544 113.40893961 142.0981706  110.73071014
 124.69906087 121.84435977 119.6771838  121.67320263 139.25586875
 107.53351106 121.74022495 149.34445846 126.74990865 140.10877018
 132.7798543  123.93062518 143.01535883 127.56323663 143.16986409
 127.17589943 123.58624826 144.46273611 143.01992809 137.58219665
 134.2815435  136.10801981 139.47270708 110.3249492  125.21347094
 126.38415202 119.37434269 111.64369361 145.66278681 146.65449935
  94.75900731 118.00846637 146.73020758 114.64742054 148.82544029
 133.45518112 146.12657303 124.56151333 124.84134047 133.44450007
 118.536368

In [62]:
#Associando os dados parar linkar com as Datas

previsoes_microsoft = pd.DataFrame({
    'Data': x_test_micro['data_inicio'], 
    'Previsao': y_pred_micro,
    'Empresas': 'Microsoft Corporation'            
})

previsoes_sony = pd.DataFrame({
    'Data': x_test['dias_desde_inicio'], 
    'Previsao': y_pred,
    'Empresas': 'Sony Corporation'  
})

previsoes_ubisoft = pd.DataFrame({
    'Data': x_test_ubi['data_inicio_ubi'], 
    'Previsao': y_pred_ubi,
    'Empresas': 'Ubisoft Entertainment'  

})

previsoes_EA = pd.DataFrame({
    'Data': x_test_EA['data_inicio_EA'], 
    'Previsao': y_pred_EA,
    'Empresas': 'Electronic Arts'  

})

#Concatenando todos os datas frames

previsoes_totais = pd.concat([previsoes_microsoft, previsoes_sony, previsoes_ubisoft, previsoes_EA])

previsoes_totais = previsoes_totais.sort_values(by = 'Data').reset_index(drop = True)



In [66]:
print(previsoes_totais['Data'].head())

0    274
1    274
2    274
3    274
4    279
Name: Data, dtype: int64


int64


TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

In [67]:
from datetime import timedelta
dias_ajustados =  pd.to_timedelta(previsoes_totais['Data'], unit='D')
nova_data = pd.Timestamp('2025-01-01')
previsoes_totais['Data'] = nova_data + dias_ajustados

In [68]:
print(previsoes_totais.head())
print(previsoes_totais.tail())
print(previsoes_totais.shape)

        Data    Previsao               Empresas
0 2025-10-02   12.483271       Sony Corporation
1 2025-10-02  147.104064  Microsoft Corporation
2 2025-10-02   96.254563        Electronic Arts
3 2025-10-02   11.208248  Ubisoft Entertainment
4 2025-10-07  149.712844  Microsoft Corporation
           Data    Previsao               Empresas
1003 2030-09-27  145.662787        Electronic Arts
1004 2030-10-22    5.063834  Ubisoft Entertainment
1005 2030-10-22   19.383156       Sony Corporation
1006 2030-10-22  137.902194        Electronic Arts
1007 2030-10-22  416.168329  Microsoft Corporation
(1008, 3)


In [69]:

novo_dataset = previsoes_totais

caminho = r"C:\Users\dante\OneDrive\Documentos\9. Meus Projetos\Projetos Power Bi\Mercado de Ações\Dados\ machine learning"

novo_dataset.to_excel(caminho, index=False, engine='openpyxl')